# Оценка эффективности каналов привлечения пользователей


## Постановка задачи

Компания по доставке продуктов на дом. Сервис доступен как в приложении на ios, так и на android. Необходимо проанализировать поведение пользователей, а также оценить эффективность каналов их привлечения. 

Данные из AppMetrica за период с 1 января по 31 марта 2020, только по пользователям, зарегистрированным позднее 1 января 2020.


## Описание данных:

| Поле               | Описание                                                   |
| :----------------  | :----------------------------------------------------------|
| `date`             | дата совершения события                                    |
| `event`            | событие:                                                   |
|                    |  * app_install – установка приложения                      |
|                    |  * app_start – открыть приложения                          |
|                    |  * registration – зарегистрироваться                       | 
|                    |  * search – перейти на страницу поиска товаров (каталог)   |
|                    |  * open_item – открыть товар                               |
|                    |  * choose_item – отправить товар в корзину                 |
|                    |  * tap_basket – перейти в корзину                          |
|                    |  * purchase – подтверждение покупки                        |
|                    |                                                            |
| `gender`           | пол пользователя                                           |
| `os_name`          | платформа пользователя                                     |
| `city`             | город пользователя                                         |
| `device_id`        | идентификатор устройства пользователя                      |
| `urm_source`       | канал, с которого пришел пользователь:                     |
|                    |  * yandex-direct – Яндекс директ                           |
|                    |  * google_ads – реклама в Google                           |
|                    |  * vk_ads – реклама в ВК                                   | 
|                    |  * instagram_ads – реклама в instagram                     |
|                    |  * facebook_ads – реклама в facebook                       |
|                    |  * referral – акция «приведи друга»                        |
|                    |  * "-" – канал не определен или это скачивание приложения напрямую или посещение не с рекламы |
| `purchase_sum`     | стоимость покупки (при совершении события `purchase`)      |


## Примечание:

- В выгрузке только уникальные действия пользователей за каждый день. 
- Можно миновать стадию установки приложения, если оно было установлено ранее.
- Можно миновать стадию регистрации, если пользователь был уже залогинен на момент сессии. Однако незарегистрированные пользователи не могут оформить покупку.

## Задачи:

**1) Посчитайте количество уникальных пользователей по месяцам (MAU).**

**2) Посчитайте количество установок по месяцам.**

**3) Присвойте пользователям когорты по дню установки приложения и посчитайте для них  конверсию из установки в покупку в течение 7 дней. Для какой когорты конверсия была наибольшей? Ответ в формате: дд.мм.гггг. Примечание: считаем пользователя сконвертировавшимся, если с момента установки до совершения первой покупки прошло не более 7 дней.**

**4) С какого платного маркетингового канала пришло больше всего новых пользователей?**

**5) Проанализируйте на каком этапе воронки отваливается большая часть зарегистрированных пользователей. На каком шаге отваливается больше всего зарегистрированных пользователей? В качестве ответа один из шагов, конверсия из которого (в следующий шаг) самая низкая, в таком формате: «Поиск», «Добавление товара», «Переход в корзину».**

**6) Пользователи, пришедшие с каких каналов, показали самую низкую конверсию в первую покупку?**

**7) Пользователи, пришедшие с какого канала, имеют медианный первый чек выше? (учитываются только первые покупки пользователей)**

**8) Данные по затратам на рекламу:**
- **Яндекс – 10 491 707 руб.**
- **Гугл – 10 534 878 руб.**
- **Фейсбук – 8 590 498 руб.**
- **Инстаграм – 8 561626 руб.**
- **ВК – 9 553 531руб.**

**Какой платный канал привлечения имеет самый высокий ROMI?**

In [1]:
# Импорт библиотек
import pandas as pd
import numpy as np

import warnings
warnings.filterwarnings('ignore')

In [2]:
# Загрузим данные, посмотрим на их размерность и отобразим первые 5 строк
df = pd.read_csv('case_data.csv', parse_dates=['date'])
print(f'Размерность данных {df.shape}')
df.head()

Размерность данных (2747968, 8)


,date,event,purchase_sum,os_name,device_id,gender,city,utm_source
0,2020-01-01,app_start,NaN,android,669460,female,Moscow,-
1,2020-01-01,app_start,NaN,ios,833621,male,Moscow,vk_ads
2,2020-01-01,app_start,NaN,android,1579237,male,Saint-Petersburg,referal
3,2020-01-01,app_start,NaN,android,1737182,female,Moscow,facebook_ads
4,2020-01-01,app_start,NaN,ios,4029024,female,Moscow,facebook_ads


In [3]:
# Посмотрим на типы данных в датафрейме
df.dtypes

date            datetime64[ns]
event                   object
purchase_sum           float64
os_name                 object
device_id                int64
gender                  object
city                    object
utm_source              object
dtype: object

In [4]:
# Посмотрим на пропущенные значения в данных
missing = df.isna().sum()
missing[missing > 0]

purchase_sum    2606585
dtype: int64

In [5]:
# Посмотрим есть ли дубликаты строк в датафрейме
df.duplicated().sum()

0

In [6]:
# Посмотрим на описательную статистику по категориальным переменным
df.describe(include=['O']).T

,count,unique,top,freq
event,2747968,7,app_start,748705
os_name,2747968,2,android,1628119
gender,2747968,2,female,1543644
city,2747968,2,Moscow,1491481
utm_source,2747968,7,-,610458


## 1. Посчитаем количество уникальных пользователей по месяцам (MAU)

In [7]:
df.set_index('date') \
    .device_id \
    .resample(rule='1m').nunique() \
    .reset_index() \
    .rename(columns={'device_id': 'count'})

,date,count
0,2020-01-31,99161
1,2020-02-29,75032
2,2020-03-31,74623


## 2. Посчитаем количество установок по месяцам

In [8]:
df.query('event == "app_install"') \
    .set_index('date') \
    .device_id \
    .resample(rule='1m').count() \
    .reset_index() \
    .rename(columns={'device_id': 'count'})

,date,count
0,2020-01-31,80297
1,2020-02-29,38078
2,2020-03-31,36222


## 3. Посчитаем конверсию из установки в покупку в течение 7 дней для дневных когорт

In [9]:
# Отфильтруем необходимые наблюдения
cohorts_df = df.query('event == "app_install" | event == "purchase"') \
    .groupby(['device_id', 'event'], as_index=False) \
    .agg({'date': 'min'}) \
    .pivot(index = 'device_id', columns = 'event', values = 'date') \
    .reset_index()

# Посчитаем количество дней с установки до покупки
cohorts_df['diff'] = (cohorts_df['purchase'] - cohorts_df['app_install']).dt.days

# Сформируем когорты
cohorts_df['cohort'] = cohorts_df.groupby('device_id').app_install.transform('min')

# Определим размер когорт
cohorts_df['cohort_size'] = cohorts_df.groupby('cohort').device_id.transform('nunique')

# Отберем наблюдения в течении первых семи дней
whithin_7_days = cohorts_df.query('diff <= 7') \
    .groupby(['cohort', 'cohort_size'], as_index=False) \
    .agg({'purchase': 'count'})

# Рассчитаем конверсию из установки в покупку
whithin_7_days['CR'] = (whithin_7_days.purchase / whithin_7_days.cohort_size).mul(100).round(1)

# Посмотрим на результат
whithin_7_days.sort_values('CR', ascending=False).head()

,cohort,cohort_size,purchase,CR
0,2020-01-01,3579.0,1408,39.3
8,2020-01-09,1424.0,558,39.2
14,2020-01-15,4310.0,1650,38.3
13,2020-01-14,5173.0,1973,38.1
1,2020-01-02,3144.0,1186,37.7


Конверсия из установки в покупку в течение 7 дней была наибольшей для пользователей из когорты `2020-01-01` - 39,3%.

## 4. Посчитаем новых пользователей с платных каналов

In [10]:
df.query('event == "app_install" & utm_source !="-"') \
    .utm_source \
    .value_counts(normalize=True).mul(100).round(2) \
    .reset_index() \
    .rename(columns={'index': 'utm_source', 'utm_source': 'percentage'})

,utm_source,percentage
0,yandex-direct,24.05
1,google_ads,21.52
2,vk_ads,18.99
3,instagram_ads,16.45
4,facebook_ads,11.39
5,referal,7.60


Больше всего новых пользователей пришло из `yandex-direct`.

## 5. Проанализируем на каком этапе воронки отваливается большая часть зарегистрированных пользователей

In [11]:
# Отберем id пользователей которые совершили покупку
purchase = df.query('purchase_sum > 0').device_id.unique().tolist()
# Отберем id пользователей которые зарегистрировались
register = df.query('event == "register"').device_id.unique().tolist()

# Объединим два списка и создадим set из уникальных id
registered_list = list(set(register + purchase))

# Отфильтруем датафрейм с пользователями которые совершили регистрацию
registered_users = df.query('device_id in @registered_list')
registered_users[:5]

,date,event,purchase_sum,os_name,device_id,gender,city,utm_source
0,2020-01-01,app_start,NaN,android,669460,female,Moscow,-
1,2020-01-01,app_start,NaN,ios,833621,male,Moscow,vk_ads
2,2020-01-01,app_start,NaN,android,1579237,male,Saint-Petersburg,referal
3,2020-01-01,app_start,NaN,android,1737182,female,Moscow,facebook_ads
5,2020-01-01,app_start,NaN,android,5148476,male,Saint-Petersburg,-


In [12]:
# Определим дату регистрации для каждого пользователя
register_date = registered_users.query('event == "register"') \
    .groupby('device_id', as_index=False) \
    .agg(register_date=('date', 'min'))

# Добавим колонку с датой регистрации к датафрейму
registered_users = pd.merge(registered_users, register_date, how='left', on='device_id')

# Добавим условие, если дата регистрации < даты совершения события то пользователь еще не зарегистрировался
registered_users['is_registered'] = np.where((registered_users.register_date - registered_users.date).dt.days > 0, 'no', 'yes')
registered_users[:5]

,date,event,purchase_sum,os_name,device_id,gender,city,utm_source,register_date,is_registered
0,2020-01-01,app_start,NaN,android,669460,female,Moscow,-,2020-01-07,no
1,2020-01-01,app_start,NaN,ios,833621,male,Moscow,vk_ads,2020-01-01,yes
2,2020-01-01,app_start,NaN,android,1579237,male,Saint-Petersburg,referal,2020-01-01,yes
3,2020-01-01,app_start,NaN,android,1737182,female,Moscow,facebook_ads,2020-01-01,yes
4,2020-01-01,app_start,NaN,android,5148476,male,Saint-Petersburg,-,2020-01-24,no


In [13]:
# Отберем только прошедших регистрацию пользователей
registered_df = registered_users.query('is_registered == "yes"') \
    .groupby('event', as_index=False) \
    .agg(count=('device_id', 'count'))

# Определим воронку действий
funnel = ['app_start', 'choose_item', 'search', 'tap_basket']

# Определим последовательность действий
mapping = {'app_start': 1, 'search': 2, 'choose_item': 3, 'tap_basket': 4}

# Рассчитаем конверсию для каждого шага воронки
registered_df = registered_df.query('event in @funnel')
registered_df['step'] = registered_df.event.map(mapping)
registered_df.sort_values('step', inplace=True)
registered_df['CR'] = (registered_df['count'].div(registered_df['count'].shift(1))).mul(100).round(2)
registered_df

,event,count,step,CR
1,app_start,519754,1,NaN
5,search,495411,2,95.32
2,choose_item,392783,3,79.28
6,tap_basket,295566,4,75.25


Таким образом, большая часть зарегистрированных пользователей отваливается на этапе перехода в корзину.

## 6. Посчитаем конверсию в первую покупку для различных каналов

In [14]:
# Посчитаем количество покупок
perchases_num = df.query('event == "purchase"') \
    .groupby('utm_source', as_index=False) \
    .agg(perchases_num=('device_id', 'nunique'))

# Посчитаем количество установок
app_install = df.query('event == "app_install"') \
    .groupby('utm_source', as_index=False) \
    .agg(app_install=('device_id', 'nunique'))

# Рассчитаем конверсию
conversion_df = app_install.merge(perchases_num, on='utm_source', how='inner')
conversion_df['CR'] = (conversion_df.perchases_num / conversion_df.app_install).mul(100).round(2)
conversion_df.sort_values('CR')

,utm_source,app_install,perchases_num,CR
6,yandex-direct,29368,12028,40.96
2,google_ads,26286,11339,43.14
0,-,32460,16598,51.13
5,vk_ads,23189,12364,53.32
3,instagram_ads,20096,10762,53.55
1,facebook_ads,13916,9017,64.80
4,referal,9282,6362,68.54


Таким образом, самая низкая конверсию в первую покупку для пользователей пришедших из "Яндекс-директ".

## 7. Посчитаем медианный чек с учетом только перовой покупки пользователей для различных каналов

In [15]:
df.query('event == "purchase" & utm_source !="-"') \
    .groupby(['device_id', 'date', 'utm_source'], as_index=False) \
    .agg({'purchase_sum': lambda x: x}) \
    .groupby('device_id', as_index=False) \
    .head(1) \
    .groupby('utm_source', as_index=False) \
    .agg(median_check=('purchase_sum', 'median')) \
    .sort_values('median_check', ascending=False)

,utm_source,median_check
3,referal,395.50
2,instagram_ads,393.50
5,yandex-direct,393.00
4,vk_ads,392.50
1,google_ads,389.50
0,facebook_ads,389.25


Таким образом, наибольший медианный чек для пользователей пришедших канала «Реферальная программа».

## 8. Посчитаем какой из платных каналов привлечения имеет самый высокий ROMI

In [16]:
spendings = {'vk_ads': 9553531, 'facebook_ads': 8590498, 'google_ads': 10534878,
             'instagram_ads': 8561626, 'yandex-direct': 10491707}

romi_df = df.groupby('utm_source', as_index=False) \
    .agg(revenue=('purchase_sum', 'sum'))

romi_df['marketing'] = romi_df.utm_source.map(spendings)
romi_df['ROMI'] = ((romi_df.revenue - romi_df.marketing) / romi_df.marketing).mul(100).round(2)
romi_df.dropna(inplace=True)
romi_df.sort_values('ROMI', ascending=False)

,utm_source,revenue,marketing,ROMI
5,vk_ads,16389652.5,9553531.0,71.56
3,instagram_ads,14546969.0,8561626.0,69.91
1,facebook_ads,12249901.0,8590498.0,42.60
6,yandex-direct,13915368.0,10491707.0,32.63
2,google_ads,12868276.0,10534878.0,22.15


Таким образом, наибольшая рентабельность рекламных кампаний у канала «ВКонтакте».

# Summary


- **Конверсия из установки в покупку в течение 7 дней была наибольшей для пользователей из когорты 2020-01-01 - 39,3%.**
- **Больше всего новых пользователей пришло из yandex-direct - 24% от всех пользователей.**
- **Большая часть зарегистрированных пользователей отваливается на этапе перехода в корзину.**
- **самая низкая конверсию в первую покупку для пользователей пришедших из "Яндекс-директ" - 40,96%.**
- **Наибольший медианный чек для пользователей пришедших канала «Реферальная программа» - 395,50 руб.**
- **Наибольшая рентабельность рекламных кампаний у канала «ВКонтакте» - 71,56%.**